## Image pre-processing

In [1]:
import os
import cv2
import math
import numpy as np

Set up image path and size of output image

In [4]:
fold_index = '01'
fddb_annotations = './FDDB-folds/FDDB-fold-' + fold_index + '-ellipseList.txt'
new_width = 20
new_height = 20
width_range = int(new_width/2)
height_range = int(new_height/2)

Crop face patches by using annotations and resize it to 20x20 by using cv2.resize() method. Crop nonface patches by randomly choosing a pacth of size 20x20 outside the region of annotations. 

In [5]:
# ref - https://github.com/cezs/FDDB/blob/master/scripts/cs_fddb_convert_to_darknet.py
with open(fddb_annotations, 'r') as annotations:
    count = 0
    for filepath in annotations:
        # make labels/<year>/<month>/<day>/big directory tree
        filepath_split = filepath.rstrip('\n').rsplit('/')
        if len(filepath_split) > 1:
            filepath_img = './dataset/originalPics/' + '/'.join(filepath_split) + '.jpg'
        # annotation modification to bounding box for each ellipse in image file
        for bbox in range(int(next(annotations))):
            # supplied values
            current_line = next(annotations)
            current_line_split = current_line.split()
            major_axis_radius = float(current_line_split[0])
            minor_axis_radius = float(current_line_split[1])
            angle = float(current_line_split[2])
            center_x = float(current_line_split[3])
            center_y = float(current_line_split[4])
            # read image with cv2 package and find image dimensions
            image = cv2.imread(filepath_img)
            img_height, img_width, c = image.shape # note that the default output of cv2.imread is height, width, layer
            # calculate bounding box of rotated ellipse
            calc_x = math.sqrt((major_axis_radius * math.cos(angle))**2 + (minor_axis_radius * math.sin(angle))**2)
            calc_y = math.sqrt((major_axis_radius * math.sin(angle))**2 + (minor_axis_radius * math.cos(angle))**2)
            # crop face from the original image
            crop_image = image[max(int(center_y-calc_y),0):min(int(center_y+calc_y),img_height),max(int(center_x-calc_x),0):min(img_width,int(center_x+calc_x)),:]
            # convert to gray image
            crop_image = cv2.cvtColor(crop_image, cv2.COLOR_BGR2GRAY)
            resized_img = cv2.resize(crop_image, (new_width, new_height), interpolation=cv2.INTER_LINEAR) # resize image to size of 20x20
            # Save the cropped image
            try:
                os.listdir('./dataset/trimmed_img/')
            except:
                os.makedirs('./dataset/trimmed_img/')
            #cv2.imwrite('./dataset/trimmed_img' + '/{}'.format(str(count) + '.jpg'), resized_img)
            
            
            # crop random non-face patch from the original image
            noface_x = np.random.randint(width_range,img_width-width_range)
            noface_y = np.random.randint(height_range,img_height-height_range)
            while (max(int(center_x-calc_x-width_range),0) <= noface_x <= min(img_width,int(center_x+calc_x+width_range))) & (max(int(center_y-calc_y-height_range),0) <= noface_y <= min(int(center_y+calc_y+height_range),img_height)):
                noface_x = np.random.randint(width_range,img_width-width_range)
                noface_y = np.random.randint(height_range,img_height-height_range)                
            non_face_image = image[(noface_y-height_range):(noface_y+height_range),(noface_x-width_range):(noface_x+width_range),:]
            # Save the non-face image
            try:
                os.listdir('./dataset/trimmed_nofaceimg/')
            except:
                os.makedirs('./dataset/trimmed_nofaceimg/')
                
            # convert to gray image
            non_face_image = cv2.cvtColor(non_face_image, cv2.COLOR_BGR2GRAY)
            cv2.imwrite('./dataset/trimmed_nofaceimg' + '/{}'.format(str(count) + '.jpg'), non_face_image)
            
            count += 1
            

AttributeError: 'NoneType' object has no attribute 'shape'

Bundle the face and nonface images and shuffle it once. Split them into train dataset and test dataset.

In [ ]:
test_data_size = 100
train_data_size = 1000

In [22]:
# ref - https://github.com/MarkPrecursor/SRCNN-keras/blob/master/prepare_data.py
# Build train and test dataset
import h5py

names_face = sorted(os.listdir('./dataset/trimmed_img/'))
names_noface = sorted(os.listdir('./dataset/trimmed_nofaceimg/'))

data_face = []
label_face = []

for name in names_face:
    fpath = './dataset/trimmed_img/' + name
    face_img = cv2.imread(fpath)

    norm_face_img = face_img.astype(np.float32) / 255. # normalize value from [0,255] to [0,1]

    label_face.append(1) # label 1 means the current image contains face
    data_face.append(norm_face_img)

data_noface = []
label_noface = []

for name in names_noface:
    fpath = './dataset/trimmed_nofaceimg/' + name
    noface_img = cv2.imread(fpath)

    norm_noface_img = noface_img.astype(np.float32) / 255. # normalize value from [0,255] to [0,1]

    label_noface.append(0) # label 0 means the current image doesn't contain face
    data_noface.append(norm_noface_img)
    
data_face = np.array(data_face, dtype=np.float32)
label_face = np.array(label_face, dtype=np.int32)
data_noface = np.array(data_noface, dtype=np.float32)
label_noface = np.array(label_noface, dtype=np.int32)


# shuffle the data and label to mix face images and nonface images
index = np.arange(len(data_face))
np.random.shuffle(index)
data_face = data_face[index]
label_face = label_face[index]

index = np.arange(len(data_noface))
np.random.shuffle(index)
data_noface = data_noface[index]
label_noface = label_noface[index]



# let first train_data_size be train data
x_train_dataset = np.concatenate((data_face[:train_data_size], data_noface[:train_data_size]), axis=0) 
y_train_dataset = np.concatenate((label_face[:train_data_size], label_noface[:train_data_size]), axis=0) 

x_shape = x_train_dataset.shape
x_train_dataset = x_train_dataset.reshape((x_shape[0],x_shape[1]*x_shape[2]*x_shape[3]))

# let last test_data_size be test data
x_test_dataset = np.concatenate((data_face[-test_data_size:], data_noface[-test_data_size:]), axis=0) 
y_test_dataset = np.concatenate((label_face[-test_data_size:], label_noface[-test_data_size:]), axis=0) 

x_shape = x_test_dataset.shape
x_test_dataset  = x_test_dataset.reshape((x_shape[0],x_shape[1]*x_shape[2]*x_shape[3]))

# save train and test dataset
with h5py.File('./dataset/train_dataset.h5', 'w') as h:
    h.create_dataset('data', data=x_train_dataset, shape=x_train_dataset.shape)
    h.create_dataset('label', data=y_train_dataset, shape=y_train_dataset.shape)

with h5py.File('./dataset/test_dataset.h5', 'w') as h:
    h.create_dataset('data', data=x_test_dataset, shape=x_test_dataset.shape)
    h.create_dataset('label', data=y_test_dataset, shape=y_test_dataset.shape)

In [23]:
# verify the shape of dataset
with h5py.File('./dataset/train_dataset.h5', 'r') as h:
    data = np.array(h.get('data'))
    label = np.array(h.get('label'))
    X_train = data
    y_train = label

print(X_train.shape, y_train.shape)

with h5py.File('./dataset/test_dataset.h5', 'r') as h:
    data = np.array(h.get('data'))
    label = np.array(h.get('label'))
    X_test = data
    y_test = label

print(X_test.shape, y_test.shape)

(2000, 1200) (2000,)
(200, 1200) (200,)
